In [22]:
import pandas as pd
import os
from openai import OpenAI
from dotenv import load_dotenv

from crewai import Agent, Task, Crew, Process
# from crewai_tools import SerperDevTool, FileReadTool, ScrapeWebsiteTool, WebsiteSearchTool
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

In [23]:
load_dotenv('.env')
os.environ['OPENAI_MODEL_NAME'] = "gpt-4o-mini"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

In [25]:
topic = input("Enter the audit topic: ")
if topic:
    print("You entered:", topic)
else:
    print("Please enter an audit task.")

You entered: baby bonus scheme


In [30]:
tool_search = SerperDevTool(n_results=5, country="sg")
tool_webscrape = ScrapeWebsiteTool()

auditor = Agent(
    role = "auditor",
    goal="Return the Audit Planning Memo on {topic} based on inputs from...",
    backstory = """You are an auditor. You need to consider the inputs from the researcher to determine what to write for the audit planning memo and risk assessment""",
    allow_delegation = True,
    max_iter = 15,
    verbose = True
)

researcher = Agent(
    role="researcher",
    goal="From the internet, research and analyse and sieve out information related to {topic} only." ,
    backstory="""As a researcher, navigating and extracting critical information is crucial. You are assisting the auditor to research on topics relevant to {topic}. 
    """,
    allow_delegation=False,
    verbose=True,
    max_iter = 15
)


task_search = Task(
    description="""Returns a JSON list of websites that contains the top 5 most relevant information, in the context of Singapore, that is related to policy and regulation on {topic} only. Ignore urls that ends with .pdf.
    """,
    expected_output="""\
    Returns websites that contain the top 5 most relavant policy and regulation information on {topic} only. The output must be in JSON format with the description as the key and url as value.""",

    agent=researcher,
    tools=[tool_search],
)


task_websitesearch = Task(
    description="""Scrape through a website for information related to {topic} only
    """,
    expected_output="""\
    Summarise the website for information related to {topic} only in the Singapore context. """,

    agent=researcher,
    tools=[tool_webscrape],
    context = [task_search]
)

task_write= Task(
    description="""\
    1. Use the content to craft an audit planning memo and risk assessment based on {topic} .
    2. Sections are properly named in an engaging manner.
    3. Proofread for grammatical errors and alignment with the common style used in audit planning memos.""",

    expected_output="""
    A well-written audit planning memo and risk assessment in markdown format. Each section should have minimum of 2 and maximum of 5 paragraphs.""",
    agent=auditor,
    context=[task_websitesearch],
    output_file="APM.md"
)




In [31]:
#creating the crew

crew = Crew(
    agents=[auditor,researcher],
    tasks=[task_search,task_websitesearch,task_write],
    verbose=True
)


2024-10-08 10:05:26,197 - 11040 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [32]:
result = crew.kickoff(inputs={"topic": topic})

 [2024-10-08 10:05:27][DEBUG]: == Working Agent: researcher
 [2024-10-08 10:05:27][INFO]: == Starting Task: Returns a JSON list of websites that contains the top 5 most relevant information, in the context of Singapore, that is related to policy and regulation on baby bonus scheme only. Ignore urls that ends with .pdf.
    


> Entering new CrewAgentExecutor chain...
I need to gather relevant information on the baby bonus scheme in Singapore, specifically focusing on policy and regulation. This will help in providing accurate insights for the auditor.

Action: Search the internet  
Action Input: {"search_query": "Singapore baby bonus scheme policy regulation"}
 


Search results: Title: Baby Bonus Scheme - Made For Families
Link: https://www.madeforfamilies.gov.sg/support-measures/raising-your-child/financial-support/baby-bonus-scheme
Snippet: Your child must obtain Singapore Citizenship before 24 months of age to be eligible for the Cash Gift, and before 12 years of age to be eligible

In [33]:
print(f"Raw Output: {result.raw}")
print(f"Token Usage: {result.token_usage}")
print("-----------------------------------------\n\n")
print(f"Tasks Output of Task 1: {result.tasks_output[0]}")
print("-----------------------------------------\n\n")

Raw Output: ```markdown
# Audit Planning Memo: Baby Bonus Scheme

## Introduction

The Baby Bonus Scheme in Singapore is a pivotal initiative aimed at alleviating the financial burdens associated with raising children. This program encourages family growth by providing financial support through two main components: the Baby Bonus Cash Gift (BBCG) and the Child Development Account (CDA). This audit planning memo seeks to outline the key aspects of the scheme while identifying potential risks and challenges that may affect its effectiveness and integrity.

## Overview of the Baby Bonus Scheme

The Baby Bonus Scheme comprises the Baby Bonus Cash Gift and the Child Development Account, both designed to provide substantial financial assistance to families. The BBCG offers cash gifts that have been recently increased to $11,000 for the first and second child and $13,000 for the third and subsequent children. Disbursement occurs in installments over the child's early years, ensuring a steady 

In [ ]:
# tool = SerperDevTool(n_results=3, country="sg")

# agent = Agent(
#     role="search agent",
#     goal="return most relevant information, in the context of Singapore, that is related to policy and regulation on {topic}" ,
#     backstory="""You're working on gathering all relavant policy and regualtion information on {topic}.
#     """,
#     allow_delegation=False,
#     verbose=True,
#     max_iter=40
# )

# task = Task(
#     description="""return most relevant information, in the context of Singapore, that is related to policy and regulation on {topic}
#     """,
#     expected_output="""\
#     A detailed report with most relevant information information that is related to policy and regulation on {topic}.""",

#     agent=agent,
#     tools=[tool],
# )

# crew = Crew(
#     agents=[agent],
#     tasks=[task],
#     verbose=True
# )

# result = crew.kickoff(inputs={"topic": topic})

In [ ]:
# print(f"Raw Output: {result.raw}")
# print(f"Token Usage: {result.token_usage}")
# print("-----------------------------------------\n\n")
# print(f"Tasks Output of Task 1: {result.tasks_output[0]}")
# print("-----------------------------------------\n\n")
# # print(f"Tasks Output of Task 2: {result.tasks_output[1]}")